## Analysis and Cross Validation

In [1]:
import math
import numpy as np
import glob
import os
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, CondensedNearestNeighbour
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, precision_recall_curve, ConfusionMatrixDisplay

import matplotlib.pyplot as plt

2022-12-04 17:47:36.576597: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-04 17:47:36.777006: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-04 17:47:37.643277: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/bscrow/miniconda3/lib/
2022-12-04 17:47:37.646418: W tensorflow/compiler/xla/stream_executo

In [2]:
tf.config.list_physical_devices('GPU')

2022-12-04 17:47:50.918772: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-04 17:47:50.962112: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-04 17:47:50.962428: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Model setup and training

In [3]:
# helper functions

# Get datasets
# ------------
def get_datasets(fname):
    sets = []
    with open(fname, 'rb') as f:
        for i in range(5):
            dynamic_X = np.load(f,allow_pickle=True)
            static_X = np.load(f,allow_pickle=True)
            dynamic_y = np.load(f,allow_pickle=True)
            sets.append([dynamic_X, static_X, dynamic_y])
    return sets


# Generating y labels
# -------------------
def is_sustained_increased_icp(vals, thres=4):
    return (vals > 22).sum(axis=1) > thres

def any_sustained_increased_icp(vals, thres=4):
    result = np.zeros((vals.shape[0], vals.shape[1]-5))
    for i in range(vals.shape[1]-5):
        result[:, i] = is_sustained_increased_icp(vals, thres)
    return np.any(result, axis=1)


# Resamplers
# ----------
def smote_resample_wrapper(dynam_x, static_x, y, random_state=100):
    # combining dynamic and static features into a 2d array with 
    # shape = (nsamples, 18 * no. of dynamic features + no of static features)
    flattened_features = dynam_x.reshape((dynam_x.shape[0], 18 * 2))
    flattened_features = np.concatenate([flattened_features, static_x], axis=1)

    sm = SMOTE(random_state=random_state)
    resampled_x, resampled_y = sm.fit_resample(flattened_features, y)

    resampled_dynam_x = resampled_x[:, :36].reshape((resampled_x.shape[0], 18, 2))
    resampled_static_x = resampled_x[:, 36:]
    return resampled_dynam_x, resampled_static_x, resampled_y
# undersample via random sampling 
def undersample_wrapper(dynam_x, static_x, y, random_state=100, count=None):
    # combining dynamic and static features into a 2d array with 
    # shape = (nsamples, 18 * no. of dynamic features + no of static features)
    flattened_features = dynam_x.reshape((dynam_x.shape[0], 18 * 2))
    flattened_features = np.concatenate([flattened_features, static_x], axis=1)
    
    sm = RandomUnderSampler(random_state=random_state, sampling_strategy=count)
    resampled_x, resampled_y = sm.fit_resample(flattened_features, y)

    resampled_dynam_x = resampled_x[:, :36].reshape((resampled_x.shape[0], 18, 2))
    resampled_static_x = resampled_x[:, 36:]
    return resampled_dynam_x, resampled_static_x, resampled_y

# Printing result metrics
# -----------------------
def print_results(y_true, y_probs):
    y_pred = np.argmax(y_probs, axis=1)
    disp = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(y_true, y_pred))
    disp.plot()
    plt.show()
    print("accuracy:", accuracy_score(y_true, y_pred))
    print("precision:", precision_score(y_true, y_pred))
    print("recall:", recall_score(y_true, y_pred))
    print("f1:", f1_score(y_true, y_pred))
    print("auc:", roc_auc_score(y_true, y_probs[:, 1]))

    precision, recall, thresholds = precision_recall_curve(y_true, y_probs[:, 1])
    plt.plot(recall, precision)
    plt.ylabel("Precision")
    plt.xlabel("Recall")
    plt.show()

In [4]:
batch_size = 512
dropout = 0.1
epochs = 30

def LSTM_model(dynamic_X, static_X):
    """
    LSTM

        - units = dimension of hidden state. Hidden state is like an output of LSTM 
        in every timestep. This means our LSTM will be outputting <units> real numbers 
        in the end. It also means, the amount of neurons in every layer of the neuron
        in LSTM is <units>. Note that in LSTM, we have the forget gate, input gate
        and output gate.

        - input_shape = (timesteps,no. of params)

        - return_sequences = True is necessary when chaining LSTM layers, so that
        the second LSTM layer has a three-dimensional sequence input
    """
    # Dynamic model
    lstm_model = Sequential()
    lstm_model.add(Input(shape=dynamic_X.shape[1:3])) # icp + rolling standard deviation
    # lstm_model.add(LayerNormalization(axis=1))
    lstm_model.add(LSTM(units=30, dropout=dropout, return_sequences=True))
    lstm_model.add(LSTM(units=10,kernel_regularizer='l2'))

    # Static model
    static_model = Sequential()
    static_model.add(Input(shape=(static_X.shape[1],)))
    static_model.add(Dense(units=20,activation='relu'))
    static_model.add(Dropout(dropout))
    static_model.add(Dense(units=10,activation='relu', kernel_regularizer='l2'))

    # Combine & softmax
    concat = Concatenate()([lstm_model.output, static_model.output])
    z = Dense(units=2)(concat)
    z = Softmax()(z)
    
    merged_model = Model(inputs=[lstm_model.input, static_model.input], outputs=z)
    opt = Adam(learning_rate=0.0025)
    merged_model.compile(optimizer=opt, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=["accuracy"])

    return merged_model

def LSTM_only_model(dynamic_X):
    # Dynamic model
    lstm_model = Sequential()
    lstm_model.add(Input(shape=dynamic_X.shape[1:3])) # icp + rolling standard deviation
    # lstm_model.add(LayerNormalization(axis=1))
    lstm_model.add(LSTM(units=30, dropout=dropout, return_sequences=True))
    lstm_model.add(LSTM(units=10,kernel_regularizer='l2'))
    lstm_model.add(Dense(units=2))
    lstm_model.add(Softmax())

    opt = Adam(learning_rate=0.0025)
    lstm_model.compile(optimizer=opt, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=["accuracy"])

    return lstm_model

In [15]:
sets = get_datasets("../dataset/train_test_allcols_label12_offset0_slide1_max100.npy")

# convert icp measurements in y to a binary prediction outcome
for i in range(5):
    sets[i][2] = any_sustained_increased_icp(sets[i][2][:,:6], thres=4).astype(int)

# get file names to figure out which feature is being used
with open("../dataset/dynam_col_names.txt", "r") as f:
    colnames = list(map(lambda x:x.strip(), f.readlines()))

for j in range(1,19):
    # select feature column
    colname = colnames[j]
    X = []
    for i in range(5):
        X.append(sets[i][0][:, :, [0,j]])

    tests_ys = []
    pred_ys = []

    for i in range(5):
        print(f"Dataset {j}: {colname}, Cross valdation fold {i+1}")
        test_dynamic_X = X[i]
        train_dynamic_X = np.concatenate([X[j] for j in range(5) if j != i], axis=0)
        test_y = sets[i][2]
        train_y = np.concatenate([sets[j][2] for j in range(5) if j != i], axis=0)
        test_static_X = sets[i][1]
        train_static_X = np.concatenate([sets[j][1] for j in range(5) if j != i], axis=0)
        
        print("Before resampling",train_dynamic_X.shape, train_static_X.shape, train_y.shape)
        # downsample negative samples to 10000
        train_dynamic_X, train_static_X, train_y = undersample_wrapper(
            train_dynamic_X, train_static_X, train_y, count={0:10000, 1:sum(train_y)}
        )
        print("After downsampling", train_dynamic_X.shape, train_static_X.shape, train_y.shape, sum(train_y) / train_y.shape[0])
        # upsample positive samples to 10000 via SMOTE
        train_dynamic_X, train_static_X, train_y = smote_resample_wrapper(
            train_dynamic_X, train_static_X, train_y
        )
        print("After smote", train_dynamic_X.shape, train_static_X.shape, train_y.shape, sum(train_y) / train_y.shape[0])
        
        model = LSTM_model(train_dynamic_X, train_static_X)
        model.fit(
            [train_dynamic_X, train_static_X], train_y, 
            epochs=epochs, batch_size=batch_size, shuffle=True, 
        )

        pred_y = model.predict([test_dynamic_X, test_static_X])
            
        # model = LSTM_only_model(train_dynamic_X[:,:,1:])
        # model.fit(
        #     train_dynamic_X[:,:,0:1], train_y, 
        #     epochs=epochs, batch_size=batch_size, shuffle=True, verbose=0
        # )

        # pred_y = model.predict(test_dynamic_X[:,:,0:1])
        # print_results(test_y, pred_y)

        with open(f'l60om100_at_least_5_out_of_6_icp_only/{colname}_down_smote_20k.npy', 'ab') as f:
            np.save(f, test_y)
            np.save(f, pred_y)

Dataset 1: stddev_roll_10, Cross valdation fold 1
Before resampling (68380, 18, 2) (68380, 3) (68380,)
After downsampling (12552, 18, 2) (12552, 3) (12552,) 0.20331421287444232
After smote (20000, 18, 2) (20000, 3) (20000,) 0.5
543/543 [==============================] - 2s 3ms/step
Dataset 1: stddev_roll_10, Cross valdation fold 2
Before resampling (68495, 18, 2) (68495, 3) (68495,)
After downsampling (12097, 18, 2) (12097, 3) (12097,) 0.1733487641564024
After smote (20000, 18, 2) (20000, 3) (20000,) 0.5
540/540 [==============================] - 2s 3ms/step
Dataset 1: stddev_roll_10, Cross valdation fold 3
Before resampling (68806, 18, 2) (68806, 3) (68806,)
After downsampling (12239, 18, 2) (12239, 3) (12239,) 0.1829397826619822
After smote (20000, 18, 2) (20000, 3) (20000,) 0.5
530/530 [==============================] - 2s 3ms/step
Dataset 1: stddev_roll_10, Cross valdation fold 4
Before resampling (68748, 18, 2) (68748, 3) (68748,)
After downsampling (12088, 18, 2) (12088, 3) (1208

In [18]:
train_dynamic_X[:,:,1:].shape

(20000, 18, 1)

## Printing model results

In [16]:
metric_mean_stds = []
for colname in colnames[1:]:
    scores = []
    with open(f'l60om100_at_least_5_out_of_6_icp_only/{colname}_down_smote_20k.npy', 'rb') as f:
        for i in range(5):
            y_true = np.load(f)
            y_probs = np.load(f)
            y_pred = np.argmax(y_probs, axis=1)
            
            accuracy = accuracy_score(y_true, y_pred)
            precision = precision_score(y_true, y_pred)
            recall = recall_score(y_true, y_pred)
            f1 = f1_score(y_true, y_pred)
            auc = roc_auc_score(y_true, y_probs[:, 1])
            
            scores.append([accuracy, precision, recall, f1, auc])
    scores = np.array(scores)
    metric_mean_stds.append(np.concatenate([np.mean(scores, axis=0), np.std(scores, axis=0)]))
metric_mean_stds = np.stack(metric_mean_stds)[:, [0,5,1,6,2,7,3,8,4,9]]

df = pd.DataFrame(
    metric_mean_stds, columns=[
        "Accuracy mean", "Accuracy std",
        "Precision mean", "Precision std",
        "Recall mean", "Recall std",
        "F1 mean", "F1 std",
        "AUC mean", "AUC std",
    ], index=[colname[:-4] for colname in colnames[1:]]
)
df.to_csv("l60om100_at_least_5_out_of_6_icp_only.csv")